### 구글 드라이브 연결

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 필수 설치 라이브러리

In [ ]:
!pip install -U langchain openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.5/257.5 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.9/260.9 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.9/70.9 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 17.5 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packagin

In [ ]:
import os
from typing import Dict, List

from langchain.chains import ConversationChain, LLMChain, LLMRouterChain
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import ChatPromptTemplate
from pydantic import BaseModel

### API 키 입력

In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = #getpass.getpass()

### LLM 파트 구현
* 게임룰에 대한 정보들을 얻는 방법을 프롬프트 체인을 이용해 구성했습니다.
* 부루마블이라는 보드게임을 진행하기위한 기본적인 rule과 건물을 지을 수 있는 규칙이 들어간 데이터를 이용해서 문답을 진행합니다.

In [ ]:
PATH = "/content/drive/MyDrive/Lecture/LLMs/LLM & Langchain/Langchain examples/Projects/Chatbots/chain_prompts" # set PATH
RULE_1 = os.path.join(
    PATH, "game_basic.txt"
)
RULE_2 = os.path.join(
    PATH, "game_building.txt"
)


def read_prompt_template(file_path: str) -> str:
    with open(file_path, "r") as f:
        prompt_template = f.read()

    return prompt_template


def create_chain(llm, template_path, output_key):
    return LLMChain(
        llm=llm,
        prompt=ChatPromptTemplate.from_template(
            template=read_prompt_template(template_path)
        ),
        output_key=output_key,
        verbose=True,
    )


llm = ChatOpenAI(temperature=0.1, max_tokens=4000, model="gpt-3.5-turbo-0125")

rule_1 = create_chain(
    llm=llm,
    template_path=RULE_1,
    output_key="text",
)
rule_2 = create_chain(
    llm=llm,
    template_path=RULE_2,
    output_key="text",
)


destinations = [
    "basic: This page describes the basic rules used to play the board game Burumble.",
    "building: This is where you'll find the rules for buildings as you play the board game.",
]
destinations = "\n".join(destinations)
router_prompt_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations)
router_prompt = PromptTemplate.from_template(
    template=router_prompt_template, output_parser=RouterOutputParser()
)
router_chain = LLMRouterChain.from_llm(llm=llm, prompt=router_prompt, verbose=True)

multi_prompt_chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains={
        "basic": rule_1,
        "building": rule_2,
    },
    default_chain=ConversationChain(llm=llm, output_key="text"),
)


class UserRequest(BaseModel):
    user_message: str


def gernerate_answer(req: UserRequest) -> Dict[str, str]:
    context = req.dict()
    context["input"] = context["user_message"]
    answer = multi_prompt_chain.run(context)

    return {"answer": answer}

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


### User 데이터 입력
* 유저 데이터 입력 후 결과를 확인 합니다.

In [ ]:
user_data = {
    "user_message": "지금 이 곳에서 어떤 건물을 지을 수 있어?"
}


In [ ]:
request_instance = UserRequest(**user_data)

In [ ]:
gernerate_answer(request_instance)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new LLMRouterChain chain...

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: 건물의 종류로는 별장, 빌딩, 호텔 3종류가 있다. 색상은 빨강, 노랑, 파랑이나 간혹 바뀔 때도 있다.[7] 이 게임의 진정한 핵심은 건물을 짓는 데 있다. 건물이 없는 땅은 통행료가 없다시피 하지만 비싼 건물을 지을수록 수익률이 기하급수적으로 늘어난다.

원래 정식 규칙에 따르면 전후반으로 나뉘며 전반에는 땅만 살 수 있고 건물은 지을 수 없다. 하지만 룰을 자세히 모르거나 알아도 번거롭고 흐름도 끊겨서 무시하고 하는 경우가 많다.

그리고 후반에 건물을 지을 때는 반드시 그 땅에 오지 않아도 자기 차례라면 자금사정이 허락하는 한 자기 땅 아무 곳에나 건물을 지을 수 있다. 이렇게 하는 사람들이 드물어서 그렇지. 그리고 이렇게 해야 '포오드 작전'이 의미가 있는 것이다. 그런데 이건 너도나도 포오드 작전을 쓰기 시작하면 밸런스가 붕괴되기 쉬워진다.
즉, 전반전에는 자기 말이 도착한 위치의 땅만 살 수 있는 데 비해 후반전에는 자기 말이 어디 있건 자기 턴는 모든 자기 땅에 어디든 건물을 짓거나 팔 수 있는 것. 제작사 공식 전략인 포오드 작전이나 링컨 작전이 이래야 성립 가능한 것이, 이런 작전은 결국 '언제 걸릴지 모르는 땅 여기저기에 별장이나 흩어 지어놓기보다는 다른 플레이어(특히 집중적으로 견제하고 공격해야 할 상대)의 말이 지나가는 앞길(2~12칸)에 똘똘한 호텔 몇개를 배치해서 걸리기 쉽게 만드는 것이 효율적이다'라는 이야기인데, 자기 땅의 건물을 자유롭게 전략적으로 재배치할수 없이 말이 도착한 위치에서만 건물을 살 수 있다면 주사위 운이 말도 안 되게 좋지 않은 한 매번 '상대가 걸릴만한 곳'에 건물을 배치하는 것은 불가능하다. 건물의 자유로운 전략적 재배치가 가능하다는 것을 전제로 성립하는 작전인 것. 그리고 이런 오리지

{'answer': '빌딩을 지을 수 있습니다.'}